In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
df = pd.read_csv("../input/tabular-playground-series-apr-2022/train.csv")
train_labels = pd.read_csv("../input/tabular-playground-series-apr-2022/train_labels.csv")

In [ ]:
df.info()

In [ ]:
#Dropping subject and step
df = df.drop(['subject','step'],axis=1)

In [ ]:
#Adding max of every sensor as new column
a = df.groupby('sequence', as_index=False).max()
#Adding min of every sensor as new column
b = df.groupby('sequence', as_index=False).min().add_suffix('b')
b.rename(columns = {'sequenceb':'sequence'}, inplace = True)
c = pd.merge(a,b,on="sequence")
#Adding standard deviation of every sensor as new column
f = df.groupby('sequence', as_index=False).std().add_suffix('e')
f.rename(columns = {'sequencee':'sequence'}, inplace = True)
g = pd.merge(c,f,on="sequence")
#Adding mean of every sensor as new column
z = df.groupby('sequence', as_index=False).mean().add_suffix('z')
z.rename(columns = {'sequencez':'sequence'}, inplace = True)
df = pd.merge(g,z,on="sequence").add_suffix('z')
df.rename(columns = {'sequencez':'sequence'}, inplace = True)


In [ ]:
#final dataset
df.info()

In [ ]:
#Merging labels and dataframe
df = pd.merge(df,train_labels,on="sequence")

In [ ]:
x_train = df.drop(['state','sequence'],axis=1)
y_train = df['state']

In [ ]:
#CatBoostClassifier
from catboost import CatBoostClassifier
clf = CatBoostClassifier(verbose=False)

In [ ]:
clf.fit(x_train,y_train)

# Pre Processing Test Data

In [ ]:
x_test = pd.read_csv("../input/tabular-playground-series-apr-2022/test.csv")
x_test = x_test.drop(['subject','step'],axis=1)

In [ ]:
a = x_test.groupby('sequence', as_index=False).max()
b = x_test.groupby('sequence', as_index=False).min().add_suffix('b')
b.rename(columns = {'sequenceb':'sequence'}, inplace = True)
c = pd.merge(a,b,on="sequence")
f = x_test.groupby('sequence', as_index=False).std().add_suffix('e')
f.rename(columns = {'sequencee':'sequence'}, inplace = True)
g = pd.merge(c,f,on="sequence")

z = x_test.groupby('sequence', as_index=False).mean().add_suffix('z')
z.rename(columns = {'sequencez':'sequence'}, inplace = True)
x_test = pd.merge(g,z,on="sequence").add_suffix('z')
x_test.rename(columns = {'sequencez':'sequence'}, inplace = True)


In [ ]:
x_test_sequence = x_test['sequence']

In [ ]:
x_test = x_test.drop(['sequence'],axis=1)

In [ ]:
#Predicting Output
predictions = clf.predict(x_test)

In [ ]:
#Saving Output
output = pd.DataFrame({'sequence': x_test_sequence, 'state': predictions})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")
